In [2]:
import warnings
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.chdir('D:/工作/data/shibie/')
warnings.filterwarnings('ignore')
train_data = pd.read_csv('train_data.csv')
test =pd.read_csv('test.csv')

In [18]:
train_data = train_data.set_index('ID')
test = test.set_index('ID')
Idx = train_data.index
data = pd.concat([train_data,test])

样本的特征中存在78个特征描述企业在年初的状况，另外78个特征描述企业在年末的状况，对于特征确实的企业的，为描述企业在年初和年末的所有特征全部缺失。

In [2]:
def FinancialFeature(data):
    train_data = data.copy()
    begin = [each for each in train_data.columns if '年初' in each]
    end = [each for each in train_data.columns if '年末' in each]

    begindata = train_data[begin]
    begindata.columns = begindata.columns.str.split('_').str[0]
    enddata = train_data[end]
    enddata.columns = enddata.columns.str.split('_').str[0]

    begindata['fsum'] = begindata.sum(axis=1)
    enddata['fsum'] = enddata.sum(axis=1)
    nofill = enddata-begindata

    nofill['null']=begindata.isnull().sum(axis=1)
    nofill['null']=nofill['null'].replace(78,1)
    return nofill
nofill = FinancialFeature(train_data)

In [3]:
def code(x):
        if x[:2] == '26':
            return int(x[2:4]) 
        elif x[:2]=='na':
            return 1
        else:
            return 3
            
def Infomation(data,mask=None):
    train_data = data.copy()
    information = ['企业类型','经营期限至','登记机关','企业状态','邮政编码',
               '核准日期','行业代码','注销时间','经营期限自','成立日期','行业门类',
                     '企业类别','管辖机关','经营范围']
    infocat =  ['企业类型','登记机关','企业状态','邮政编码',
                '行业代码','行业门类','企业类别','管辖机关','经营范围']

    info = train_data[information]
    info['经营期限至'] = pd.isnull(info['经营期限至']).astype(int)
    info['注销时间'] = pd.isnull(info['注销时间']).astype(int)

    info['核准经营'] = (info['核准日期'] == info['经营期限自']).astype(int)
    info['核准成立'] = (info['核准日期'] == info['成立日期']).astype(int)
    zero = info[['核准日期','经营期限自','成立日期']].isnull().sum(axis=1)
    zero_ = zero.replace([1,2,3],1)
    info['核经营成立'] = zero_
    info = info.drop(['核准日期','经营期限自','成立日期'],axis=1)

    info['邮政编码']=info['邮政编码'].astype(str).map(code)

    span = info['经营范围'].str[1:-1].str.split(',').map(len)
    span = pd.qcut(span,10,duplicates='drop')
    span.cat.rename_categories({
        key:value for key, value in zip(span.cat.categories,range(9))})
    info['经营范围'] = span.cat.rename_categories({
        key:value for key, value in zip(span.cat.categories,range(9))}).astype(int)

    encoding = []
    for each in infocat:
        encoding.append(pd.get_dummies(info[each], prefix=each, 
                                       dummy_na=True))
    encode = pd.concat(encoding,axis=1)
    info = info.drop(infocat,axis=1)
    if mask is None:
        m = lgb.LGBMClassifier()
        m.fit(encode.to_numpy(),train_data['Label'])
        encode = encode.iloc[:, m.feature_importances_!=0]
        mask = encode.columns
    else:
        encode = encode.loc[:, mask]
    return encode, info, mask
import lightgbm as lgb
encode, info,mask = Infomation(train_data)

In [4]:
def Tax(data):
    train_data = data.copy()
    tax=['增值税','企业所得税','印花税','教育费','城建税', '注册资本']
    traintax = train_data[tax]
    for each in traintax:
        traintax['is_'+each] = pd.isnull(traintax[each]).astype(int)
    traintax.fillna(0)
    traintax['zero']=(traintax==0).sum(axis=1)
    traintax['taxsum'] = traintax.iloc[:,:5].sum(axis=1)
    traintax['taxmean'] = traintax.mean(axis=1)
    traintax['median'] = traintax.median(axis=1)
    return traintax
traintax = Tax(train_data)

In [6]:
nofill_ = nofill.fillna(-1)

In [8]:
traintax_ = traintax.fillna(-1)

In [17]:
from sklearn.model_selection import cross_val_score,StratifiedKFold

In [19]:
cv = StratifiedKFold(5,shuffle=True,random_state=10)
result = []
for tid, vid in cv.split(train_data,train_data['Label']):
    data = pd.DataFrame(index=vid)
    test2 = pd.DataFrame(index=test['ID'])
    nofillt, nofillv = nofill_.loc[tid,:], nofill_.loc[vid,:]
    traintaxt, traintaxv = traintax_.loc[tid,:], traintax_.loc[vid,:]
    infot, infov = info.loc[tid,:], info.loc[vid,:]
    encodet, encodev = encode.loc[tid,:], encode.loc[vid,:]
    yt, yv = train_data.loc[tid,'Label'], train_data.loc[vid,'Label']
    data['ynofill'],test2['ynofill'] = models(nofillt, yt, nofillv,testfill_)
    data['ytax'],test2['ytax'] =models(traintaxt, yt, traintaxv, testtax_)
    data['yinfo'],test2['yinfo'] = models(infot, yt, infov,testinfo)
    data['yencode'],test2['yencode'] = models(encodet, yt, encodev,testencode)
    model = LogisticRegression()
    model.fit(data.to_numpy(), yv)
    result.append(model.predict_proba(test2.to_numpy())[:,1])

In [20]:
pred = np.concatenate(result).reshape(5,-1).mean(axis=0)
test['Label'] = pred

In [18]:
from sklearn.linear_model import LogisticRegression

In [21]:
sub = test[['ID','Label']]

In [22]:
sub.to_csv('subm.csv',index=False)

In [10]:
testfill = FinancialFeature(test)
testtax = Tax(test)
testencode, testinfo,mask = Infomation(test,mask)

In [12]:
testfill_ = testfill.fillna(-1)
testtax_  = testtax.fillna(-1)

In [14]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

In [11]:
def models(X, y, Xv,test):
    model = lgb.LGBMClassifier()
    params = model.get_params()
    params['metric'] = 'auc'
    trainset = lgb.Dataset(X.to_numpy(), label=y)
    z = lgb.cv(params, trainset,num_boost_round=200, 
               nfold=5, stratified=True)
    n_estimators = np.array(z['auc-mean']).argmax()
    model.set_params(n_estimators=max(n_estimators,3))
    model.fit(X.to_numpy(), y)
    ypred = model.predict_proba(Xv.to_numpy())[:,1]
    ytest = model.predict_proba(test.to_numpy())[:,1]
    return ypred, ytest

In [ ]:
def submission(path):
    sub = pd.read_csv('submission1.csv')
    sub.columns=['ID','Label']
    sub['ID'] = test['ID']
    sub.to_csv('sub.csv',index=False)